<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_convnets/class_03/12%20-%20Atividade%20Avaliativa/C2W3_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Aprendizagem por transferência(_Transfer Learning_)

 Nesta atividade, você usará uma técnica chamada "Aprendizagem por transferência", na qual você utiliza uma rede já treinada para ajudá-lo a resolver um problema semelhante àquele para o qual ela foi originalmente treinada.

Vamos começar!

In [ ]:
import os
import zipfile
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img

## Conjunto de dados

Para esta tarefa, você usará o conjunto de dados `Horse or Human`, que contém imagens de cavalos e humanos. 

Faça download dos conjuntos de `treinamento` e `validação` executando a célula abaixo:

In [ ]:
# Obter o conjunto de dados de treinamento Horse ou Human
!wget -q -P /content/ https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

# Obter o conjunto de dados de validação Horse ou Human
!wget -q -P /content/ https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip

test_local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(test_local_zip, 'r')
zip_ref.extractall('/tmp/training')

val_local_zip = './validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(val_local_zip, 'r')
zip_ref.extractall('/tmp/validation')

zip_ref.close()

Esse conjunto de dados já tem uma estrutura compatível com o `flow_from_directory` do Keras, portanto, você não precisa mover as imagens para subdiretórios como fez em uma atividade avaliativa anterior.

No entanto, ainda é uma boa ideia salvar os caminhos das imagens para que você possa usá-los mais tarde:

In [ ]:
# Definir os diretórios de base de treinamento e validação
train_dir = '/tmp/training'
validation_dir = '/tmp/validation'

# Diretório com fotos de cavalos de treinamento
train_horses_dir = os.path.join(train_dir, 'horses')
# Diretório com fotos de humanos de treinamento
train_humans_dir = os.path.join(train_dir, 'humans')
# Diretório com imagens de cavalos de validação
validation_horses_dir = os.path.join(validation_dir, 'horses')
# Diretório com imagens humanas de validação
validation_humans_dir = os.path.join(validation_dir, 'humans')

# Verificar o número de imagens para cada classe e definir
print(f"Há {len(os.listdir(train_horses_dir))} imagens de cavalos para treinamento.\n")
print(f"Há {len(os.listdir(train_humans_dir))} imagens de humanos para treinamento.\n")
print(f"Há {len(os.listdir(validation_horses_dir))} imagens de cavalos para validação.\n")
print(f"Há {len(os.listdir(validation_humans_dir))} imagens de humanos para validação.\n")

Agora, dê uma olhada em uma imagem de amostra de cada uma das classes:

In [ ]:
print("Exemplo de imagem de cavalo:")
plt.imshow(load_img(f"{os.path.join(train_horses_dir, os.listdir(train_horses_dir)[0])}"))
plt.show()

print("\nExemplo de imagem humana:")
plt.imshow(load_img(f"{os.path.join(train_humans_dir, os.listdir(train_humans_dir)[0])}"))
plt.show()

O `matplotlib` facilita a visualização de que essas imagens têm uma resolução de 300x300 e são coloridas, mas você pode verificar isso usando o código abaixo:

In [ ]:
# Carregue o primeiro exemplo de um cavalo
sample_image  = load_img(f"{os.path.join(train_horses_dir, os.listdir(train_horses_dir)[0])}")

# Converta a imagem em sua representação de matriz numérica
sample_array = img_to_array(sample_image)

print(f"Cada imagem tem uma forma: {sample_array.shape}")

Como esperado, a imagem de amostra tem uma resolução de 300x300 e a última dimensão é usada para cada um dos canais RGB para representar a cor.

## Geradores de treinamento e validação

Agora que você conhece as imagens com as quais está lidando, é hora de codificar os geradores que alimentarão essas imagens na sua rede. Para isso, complete a função `train_val_generators` abaixo:

**Observação importante:** As imagens têm uma resolução de 300x300, mas o método `flow_from_directory` que você usará permite que você defina uma resolução de destino. Nesse caso, **configure um `target_size` de (150, 150)**. Isso reduzirá bastante o número de parâmetros treináveis em sua rede final, proporcionando tempos de treinamento muito mais rápidos sem comprometer a acurácia!

In [ ]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  """
  Cria os geradores de dados de treinamento e validação
  
  Args:
    TRAINING_DIR (string): caminho do diretório que contém as imagens de treinamento
    VALIDATION_DIR (string): caminho do diretório que contém as imagens de teste/validação
    
  Retorna:
    train_generator, validation_generator: tupla contendo os geradores
  """
  ### COMECE SEU CÓDIGO AQUI

  # Instanciar a classe ImageDataGenerator 
  # Não se esqueça de normalizar os valores de pixel e definir argumentos para aumentar as imagens 

  train_datagen = None

  # Passe os argumentos apropriados para o método flow_from_directory
  train_generator = train_datagen.flow_from_directory(directory=None,
                                                      batch_size=32, 
                                                      class_mode=None,
                                                      target_size=(None, None))

  # Instanciar a classe ImageDataGenerator (não se esqueça de definir o argumento rescale)
  # Lembre-se de que os dados de validação não devem ser aumentados
  validation_datagen = None

  # Passe os argumentos apropriados para o método flow_from_directory
  validation_generator = validation_datagen.flow_from_directory(directory=None,
                                                                batch_size=32, 
                                                                class_mode=None,
                                                                target_size=(None, None))
  ### TERMINE SEU CÓDIGO AQUI
  return train_generator, validation_generator

In [ ]:
# Teste seus geradores
train_generator, validation_generator = train_val_generators(train_dir, validation_dir)

**Saída Esperada:**
```
Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
```

## Aprendizagem por transferência - Crie o modelo pré-treinado

Faça o download dos pesos do `inception V3` para o diretório `/tmp/`:

In [ ]:
# Faça o download dos pesos v3 iniciais
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

Agora, carregue o modelo `InceptionV3` e salve o caminho para os pesos que você acabou de baixar:

In [ ]:
# Importar o modelo inicial  
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Criar uma instância do modelo inicial a partir dos pesos locais pré-treinados
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

Complete a função `create_pre_trained_model` abaixo.

Você deve especificar a `input_shape` correta para o modelo (lembre-se de que você definiu uma nova resolução para as imagens em vez da nativa 300x300) e tornar todas as camadas não treináveis:

In [ ]:
def create_pre_trained_model(local_weights_file):
  """
  Inicializa um modelo InceptionV3.
  
  Args:
    local_weights_file (string): caminho que aponta para um arquivo H5 de pesos pré-treinados
    
  Retorna:
    pre_trained_model: o modelo InceptionV3 inicializado
  """
  ### COMECE SEU CÓDIGO AQUI
  pre_trained_model = InceptionV3(input_shape = (None, None, None),
                                  include_top = False, 
                                  weights = None) 

  pre_trained_model.load_weights(local_weights_file)

  # Tornar todas as camadas do modelo pré-treinado não treináveis
  for None in None:
    None = None

  ### TERMINE SEU CÓDIGO AQUI

  return pre_trained_model

Verifique se tudo correu bem, comparando as últimas linhas do resumo do modelo com o resultado esperado:

In [ ]:
pre_trained_model = create_pre_trained_model(local_weights_file)

# Imprimir o resumo do modelo
pre_trained_model.summary()

**Saída Esperada:**
```
batch_normalization_v1_281 (Bat (None, 3, 3, 192)    576         conv2d_281[0][0]                 
__________________________________________________________________________________________________
activation_273 (Activation)     (None, 3, 3, 320)    0           batch_normalization_v1_273[0][0] 
__________________________________________________________________________________________________
mixed9_1 (Concatenate)          (None, 3, 3, 768)    0           activation_275[0][0]             
                                                                activation_276[0][0]             
__________________________________________________________________________________________________
concatenate_5 (Concatenate)     (None, 3, 3, 768)    0           activation_279[0][0]             
                                                                activation_280[0][0]             
__________________________________________________________________________________________________
activation_281 (Activation)     (None, 3, 3, 192)    0           batch_normalization_v1_281[0][0] 
__________________________________________________________________________________________________
mixed10 (Concatenate)           (None, 3, 3, 2048)   0           activation_273[0][0]             
                                                                mixed9_1[0][0]                   
                                                                concatenate_5[0][0]              
                                                                activation_281[0][0]             
==================================================================================================
Total params: 21,802,784
Trainable params: 0
Non-trainable params: 21,802,784


```

Para verificar se todas as camadas do modelo foram definidas como não treináveis, você também pode executar a célula abaixo:

In [ ]:
total_params = pre_trained_model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in pre_trained_model.trainable_weights])

print(f"Há um total de {total_params:,} parâmetros nesse modelo.")
print(f"Há {num_trainable_params:,} parâmetros treináveis nesse modelo.")

**Saída Esperada:**
```
Há um total de 21,802,784 parâmetros nesse modelo.
Há 0 parâmetros treináveis nesse modelo.
```

## Criando callbacks para depois

Você já trabalhou com _callbacks_ antes, portanto, o _callbacks_ para interromper o treinamento quando for atingida uma precisão de 99,9% é fornecido para você:

In [ ]:
# Defina uma classe de retorno de chamada que interrompa o treinamento quando a precisão atingir 99,9%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.999):
      print("\nAtingi 99,9% de precisão, portanto, estou cancelando o treinamento!!")
      self.model.stop_training = True

## _Pipelining_ do modelo pré-treinado com o seu próprio modelo

Agora que o modelo pré-treinado está pronto, você precisa "colá-lo" ao seu próprio modelo para resolver a tarefa em questão.

Para isso, você precisará da última saída do modelo pré-treinado, pois ela será a entrada do seu modelo. Complete a função `output_of_last_layer` abaixo.

**Nota:** Para fins de avaliação nessa atividade, use a camada `mixed7` como a última camada do modelo pré-treinado. No entanto, após o envio, sinta-se à vontade para voltar aqui e usar camadas para ver os resultados.

In [ ]:
def output_of_last_layer(pre_trained_model):
  """
  Obtém a saída da última camada de um modelo
  
  Args:
    pre_trained_model (tf.keras Model): modelo para obter a saída da última camada
    
  Retorna:
    last_output: saída da última camada do modelo 
  """
  ### COMECE SEU CÓDIGO AQUI
  last_desired_layer = None
  print('formato da última camada: ', last_desired_layer.output_shape)
  last_output = None
  print('saída da última camada: ', last_output)
  ### TERMINE SEU CÓDIGO AQUI

  return last_output

Verifique se tudo está funcionando como esperado:

In [ ]:
last_output = output_of_last_layer(pre_trained_model)

**Saída Esperada (se a camada `mixed7` foi usada):**
```
formato da última camada:  (None, 7, 7, 768)
saída da última camada:  KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 768), dtype=tf.float32, name=None), name='mixed7/concat:0', description="created by layer 'mixed7'")
```

Agora você criará o modelo final adicionando algumas camadas adicionais sobre o modelo pré-treinado.

Complete a função `create_final_model` abaixo. Você precisará usar a [Functional API](https://www.tensorflow.org/guide/keras/functional) do Tensorflow para isso, pois o modelo pré-treinado foi criado com ela. 

Vamos verificar isso primeiro:

In [ ]:
# Imprimir o tipo do modelo pré-treinado
print(f"O modelo pré-treinado tem o tipo: {type(pre_trained_model)}")

Para criar o modelo final, você usará a classe Model do Keras, definindo as entradas e saídas apropriadas, conforme descrito na primeira maneira de instanciar um modelo na [documentação](https://www.tensorflow.org/api_docs/python/tf/keras/Model).

Observe que você pode obter a entrada de qualquer modelo existente usando seu atributo `input` e, usando a API Funcional, pode usar a última camada diretamente como saída ao criar o modelo final.

In [ ]:
def create_final_model(pre_trained_model, last_output):
  """
  Anexa um modelo personalizado a um modelo pré-treinado
  
  Args:
    pre_trained_model (tf.keras Model): modelo que aceitará as entradas de treinamento/teste
    last_output (tensor): saída da última camada do modelo pré-treinado
    
  Retorna:
    model: o modelo combinado
  """
  # Achatar a camada de saída para uma dimensão
  x = layers.Flatten()(last_output)

  ### COMECE SEU CÓDIGO AQUI

  # Adicione uma camada totalmente conectada com 1024 unidades ocultas e ativação ReLU
  x = None
  # Adicionar uma taxa de dropout de 0,2
  x = None  
  # Adicionar uma camada sigmoide final para classificação
  x = None        

  # Criar o modelo completo usando a classe Model
  model = Model(inputs=None, outputs=None)

  # Compilar o modelo
  model.compile(optimizer = RMSprop(learning_rate=0.0001), 
                loss = None,
                metrics = [None])

  ### TERMINE SEU CÓDIGO AQUI
  
  return model

In [ ]:

# Salve seu modelo em uma variável
model = create_final_model(pre_trained_model, last_output)

# Inspecionar parâmetros
total_params = model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in model.trainable_weights])

print(f"Há um total de {total_params:,} parâmetros nesse modelo.")
print(f"Há {num_trainable_params:,} parâmetros treináveis nesse modelo.")

**Expected Output:**
```
Há um total de 47,512,481 parâmetros nesse modelo.
Há 38,537,217 parâmetros treináveis nesse modelo.
```

É muito parâmetros, né?!

Depois de enviar seu trabalho mais tarde, tente executar novamente esse notebook, mas use a resolução original de 300x300; você ficará surpreso ao ver quantos parâmetros a mais existem nesse caso.

Agora treine o modelo:

In [ ]:
# Execute isso e veja quantas épocas devem ser necessárias antes que a chamada de retorno inicie
#  e interrompa o treinamento com 99,9% de precisão
# (Deve levar algumas épocas)
callbacks = myCallback()
history = model.fit(train_generator,
                    validation_data = validation_generator,
                    epochs = 100,
                    verbose = 2,
                    callbacks=callbacks)

O treinamento deveria ter sido interrompido após menos de 10 épocas e deveria ter atingido uma precisão superior a 99,9% (disparando a chamada de retorno).

Isso aconteceu tão rapidamente devido ao modelo pré-treinado que você usou, que já continha informações para classificar humanos e cavalos. Lega, não acha! :-)

Agora, dê uma olhada rápida nas acurácias de treinamento e validação para cada época de treinamento:

In [ ]:
# Plote as precisões de treinamento e validação para cada época

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Acurácia de Treino')
plt.plot(epochs, val_acc, 'b', label='Acurácia de Validação')
plt.title('Acurácia de Treino e Validação')
plt.legend(loc=0)
plt.figure()

plt.show()

**Parabéns por terminar a tarefa!**

Você implementou com sucesso uma rede neural convolucional que utiliza uma rede pré-treinada para ajudá-lo a resolver o problema de classificação de humanos e cavalos.